### Task 2

#### Geospatial Sentiment Analysis Using Social Media Data

In [ ]:
# installing textblob if not available

!pip install textblob

In [ ]:
# importing necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
import geopandas as gpd
import plotly.express as px
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut

In [ ]:
# reading bitcoin_tweets.csv dataset in bit dataframe
bit = pd.read_csv('Bitcoin_tweets.csv')
bit.head()

<ipython-input-3-f1abdaf3f2d3>:2: DtypeWarning: Columns (5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bit = pd.read_csv('Bitcoin_tweets.csv')


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


### Task 2.1:

##### Data Pre-processing

In [ ]:
# checking for the unique values in columns that showed warning for mixed types

print(bit['user_followers'].unique())
print(bit['user_friends'].unique())
print(bit['user_favourites'].unique())
print(bit['user_verified'].unique())
print(bit['is_retweet'].unique())

[ 8534.  6769.   128. ...  8231. 14141.  2376.]
['7605' '1532' '332' ... 10570.0 14604.0 2898.0]
['4838' '25483' '924' ... 3067.0 42065.0 58930.0]
['False' 'True'
 "@krakenfx  #ETH #BTC  If you want to become poor, please don't buy #Bitcoin Obviously not financial advice. https://t.co/RstLTzN2MX"
 False True
 "Official ESHOP Airdrop. If You Missed Meme Token Rally, Don't Miss ESHOP.  Join Now: https://t.co/2dtf2Pqc55   Join our Telegram: https://t.co/aaOlDav2rT   @EShop_Token @elonmusk #cryptocurrency #BSC #Bitcoin #Ethereum #ETH #Airdrop #bounty #ESHOP  #Airdrops"
 '@pufferswap Nice project\n\n@karnoto_hendrik \n@ayubcandra12 \n@put_nab \n\n#YieldFarming #Airdrop #Binance #Bitcoin #pancakeswap #BNB #cryptocurrency #DeFi #BTC #BinanceSmartChain #BSC\n#pufferswap #DeFi #bsc #bnb #bitcoin #cryto #Airdrop #Airdrop']
[False nan]


In [ ]:
# converting user_followers, user_friends, user_favourites types to numeric

bit['user_followers'] = pd.to_numeric(bit['user_followers'], errors='coerce')
bit['user_friends'] = pd.to_numeric(bit['user_friends'], errors='coerce')
bit['user_favourites'] = pd.to_numeric(bit['user_favourites'], errors='coerce')

In [ ]:
# converting user_verified, is_retweet to boolean types

bit['user_verified'] = bit['user_verified'].astype(bool)
bit['is_retweet'] = bit['is_retweet'].astype(bool)

In [ ]:
# checking for null values

print(bit.isnull().sum())

user_name               4
user_location       95085
user_description    17264
user_created            0
user_followers          0
user_friends            3
user_favourites         3
user_verified           0
date                    0
text                    1
hashtags            16750
source               3318
is_retweet              0
dtype: int64


In [ ]:
# data with null text and user_location are not usefull for analysis, so they are removed from the dataset.

bit = bit.dropna(subset=['text','user_location'])

In [ ]:
# other null values in textual column are filled with unknown, no hashtags, no description

bit['user_description'].fillna('No description', inplace=True)
bit['hashtags'].fillna('No hashtags', inplace=True)
bit['source'].fillna('Unknown', inplace=True)

<ipython-input-9-8d4ae4dcf674>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit['user_description'].fillna('No description', inplace=True)


In [ ]:
# the null values in numerical columns are filled with 0

bit.fillna(0, inplace=True)

In [ ]:
# checking for null values

bit.isnull().sum()

,0
user_name,0
user_location,0
user_description,0
user_created,0
user_followers,0
user_friends,0
user_favourites,0
user_verified,0
date,0
text,0


In [ ]:
# Select 500 tweets randomly

bit_samp = bit.sample(n=500)

In [ ]:
# cleaning the user_location column to remove emojis, symbols using for better sentiment analysis

def clean_user_location(location):
    # Convert to lowercase
    location = str(location).lower()
    # Remove emojis
    location = location.encode('ascii', 'ignore').decode('ascii')
    # Remove symbols and non-alphanumeric characters
    location = re.sub(r'[^a-zA-Z0-9\s]', '', location)
    # Replace multiple spaces with a single space
    location = re.sub(r'\s+', ' ', location)
    # Remove leading and trailing whitespaces
    location = location.strip()
    return location

# Apply the cleaning function to the 'user_location' column
bit_samp['cleaned_user_location'] = bit_samp['user_location'].apply(clean_user_location)

In [ ]:
# copying the cleaned_user_location to user_location column and removing the cleaned_user_location column

bit_samp['user_location'] = bit_samp['cleaned_user_location']
bit_samp.drop(columns='cleaned_user_location',inplace=True)

In [ ]:
# cleaning the text column to remove emojis, symbols using nltk for better sentiment analysis

nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove user mentions
    text = re.sub(r"@\S+", "", text)
    # Remove emojis
    text = text.encode('ascii', 'ignore').decode('ascii')
    # Remove symbols and numbers
    text = re.sub(r"[^a-zA-Z]", " ", text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Join the words back into a string
    cleaned_text = " ".join(words)

    return cleaned_text

# Apply the cleaning function to the 'text' column
bit_samp['cleaned_text'] = bit_samp['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# copying the cleaned text to text column and removing the cleaned text column

bit_samp['text'] = bit_samp['cleaned_text']

bit_samp.drop(columns='cleaned_text',inplace=True)

### Task 2.2

##### Geocoding

In [ ]:
# creating a Nominatim geolocator object using geopy

geolocator = Nominatim(user_agent="geo_analysis")

In [ ]:
# creating a rate-limited geolocator, applying it on user_location and extracting latitude and longitude from the location column

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

bit_samp['location'] = bit_samp['user_location'].apply(geocode)
bit_samp['Lat'] = bit_samp['location'].apply(lambda x: x.latitude if x else None)
bit_samp['Lon'] = bit_samp['location'].apply(lambda x: x.longitude if x else None)

bit_samp

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,location,Lat,Lon
40306,LolitaNola,united states,stock trader-earth dweller-truth seeker- wears...,2011-01-15 18:07:29,1333.0,1191.0,210.0,True,2021-02-22 18:03:12,follow let hunt bitcoins together get exciting...,No hashtags,Twitter Web App,False,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882
155482,Sir Chasington💎🤲,utah,"Software engineer, code. #Bitcoin. Golf, gym, ...",2011-12-28 04:58:43,637.0,274.0,2735.0,True,2021-06-23 01:42:21,bitcoin token matches btc price buy bitcoin,"['Bitcoin', 'Bitcoin']",Twitter for iPhone,False,"(Utah, United States, (39.4225192, -111.714358))",39.422519,-111.714358
147867,bit.trail,gothem,Follow the money 🚀. #bitcoin #btc #eth #crypto...,2011-04-27 06:25:55,2549.0,2563.0,735.0,True,2021-06-23 08:05:04,analyst bitcoin cryptos want get rich need alt...,"['Altcoins', 'Bitcoin', 'BTCTradingView', 'DeFi']",dlvr.it,False,"(Gothem, Gotlands kommun, Gotlands län, 624 30...",57.577952,18.730293
156952,Galen Briggs,montrose pa,∞/21M… Retire early with me. “100% balls deep ...,2017-02-19 21:59:27,30.0,68.0,665.0,True,2021-06-23 00:40:59,anyone ever calculated many bitcoin actually g...,['BITCOIN'],Twitter for iPhone,False,"(Montrose, Susquehanna County, Pennsylvania, U...",41.833965,-75.877139
104315,Crypto News Exchange | #CNE | #CryptoNews,moon,"John Klerm, #Crypto #Alchemist 🧙‍♂️\n\nOwner o...",2020-05-19 09:03:12,648.0,179.0,23361.0,False,2021-04-18 09:50:00,vertex market p p trading platform low fees do...,"['doge', 'dogecoin', 'bitcoin', 'crypto', 'btc...",Twitter Web App,False,"(Muhu, Saare maakond, Eesti, (58.5959044, 23.2...",58.595904,23.219646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151703,Amber Twemlow (Twem)🇬🇧💂🏻👩🏼‍🎓❤️🌈😇🧜🏻‍♀️,point clear england,I am the Siren. Clacton/Tendring happenings. I...,2019-02-19 00:29:07,4569.0,4057.0,135649.0,True,2021-06-23 04:57:39,bitcoin btc current price gbp like updates tip...,"['Bitcoin', 'BTC']",Twemzy,False,"(Point Clear, Tendring, Essex, England, CO16 8...",51.791525,1.049376
22992,Holborn Assets,dubai united arab emirates,Our aim is to help individuals and businesses ...,2010-07-07 15:31:02,6998.0,104.0,384.0,True,2021-02-15 14:58:13,bitcoins price quadrupled since start year bit...,"['Bitcoin', 'BTC']",Twitter Web App,False,"(دبي, الإمارات العربية المتحدة, (25.0742823499...",25.074282,55.188539
206207,Crypto-buddy,united kingdom,#TRX #ETH #BTT #HEX 🐙 #oilyoctopus #bnb #bsc...,2020-01-19 00:12:17,228.0,361.0,967.0,False,2021-06-21 22:26:45,holders sneaking like tg bsc bnb btc bitcoin b...,"['bsc', 'bnb', 'btc', 'Bitcoin', 'BUSD', 'Bina...",Twitter Web App,False,"(United Kingdom, (54.7023545, -3.2765753))",54.702354,-3.276575
121062,Crypto: The Future Of Money,united states,“Do not store up for yourselves treasures on e...,2010-01-22 02:54:29,25712.0,7.0,1109.0,False,2021-05-26 21:56:35,shiba inu reach invest eth ethereum altcoins c...,"['eth', 'ethereum', 'altcoins', 'coins', 'etf'...",Twitter Web App,False,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882


In [ ]:
# drop null values in the location column

bit_samp = bit_samp.dropna(subset=['location'])

In [ ]:
# checking for null values

bit_samp.isna().sum()

,0
user_name,0
user_location,0
user_description,0
user_created,0
user_followers,0
user_friends,0
user_favourites,0
user_verified,0
date,0
text,0


### Task 2.3

###### Polarity Analysis

In [ ]:
# write a function to generate polarity

def getTextPolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [ ]:
# applying above polarity function on text column

bit_samp['polarity'] = bit_samp['text'].apply(getTextPolarity)
bit_samp.head()

<ipython-input-22-c37799c82813>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bit_samp['polarity'] = bit_samp['text'].apply(getTextPolarity)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,location,Lat,Lon,polarity
40306,LolitaNola,united states,stock trader-earth dweller-truth seeker- wears...,2011-01-15 18:07:29,1333.0,1191.0,210.0,True,2021-02-22 18:03:12,follow let hunt bitcoins together get exciting...,No hashtags,Twitter Web App,False,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,0.300
155482,Sir Chasington💎🤲,utah,"Software engineer, code. #Bitcoin. Golf, gym, ...",2011-12-28 04:58:43,637.0,274.0,2735.0,True,2021-06-23 01:42:21,bitcoin token matches btc price buy bitcoin,"['Bitcoin', 'Bitcoin']",Twitter for iPhone,False,"(Utah, United States, (39.4225192, -111.714358))",39.422519,-111.714358,0.000
147867,bit.trail,gothem,Follow the money 🚀. #bitcoin #btc #eth #crypto...,2011-04-27 06:25:55,2549.0,2563.0,735.0,True,2021-06-23 08:05:04,analyst bitcoin cryptos want get rich need alt...,"['Altcoins', 'Bitcoin', 'BTCTradingView', 'DeFi']",dlvr.it,False,"(Gothem, Gotlands kommun, Gotlands län, 624 30...",57.577952,18.730293,0.375
156952,Galen Briggs,montrose pa,∞/21M… Retire early with me. “100% balls deep ...,2017-02-19 21:59:27,30.0,68.0,665.0,True,2021-06-23 00:40:59,anyone ever calculated many bitcoin actually g...,['BITCOIN'],Twitter for iPhone,False,"(Montrose, Susquehanna County, Pennsylvania, U...",41.833965,-75.877139,0.250
104315,Crypto News Exchange | #CNE | #CryptoNews,moon,"John Klerm, #Crypto #Alchemist 🧙‍♂️\n\nOwner o...",2020-05-19 09:03:12,648.0,179.0,23361.0,False,2021-04-18 09:50:00,vertex market p p trading platform low fees do...,"['doge', 'dogecoin', 'bitcoin', 'crypto', 'btc...",Twitter Web App,False,"(Muhu, Saare maakond, Eesti, (58.5959044, 23.2...",58.595904,23.219646,0.200


In [ ]:
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
bit_samp_copy = bit_samp.copy()

# Group by geographical location and calculate the average polarity
avg_polarity_df = bit_samp_copy.groupby(['Lat', 'Lon'])['polarity'].mean().reset_index()

# Convert Lat and Lon to strings to create a hashable 'location' tuple
avg_polarity_df['location'] = avg_polarity_df[['Lat', 'Lon']].astype(str).apply(tuple, axis=1)

# Function to get country name from coordinates
def get_country(lat, lon):
    geolocator = Nominatim(user_agent="geo_analysis")
    location = None
    try:
        location = geolocator.reverse((lat, lon), language='en')
    except GeocoderTimedOut:
        pass

    return location.address.split(",")[-1].strip() if location else None

# Apply the function to create 'Country' column
avg_polarity_df['Country'] = avg_polarity_df.apply(lambda row: get_country(row['Lat'], row['Lon']), axis=1)

# Create a choropleth map for polarity
fig = px.scatter_geo(avg_polarity_df,
                     lat='Lat',
                     lon='Lon',
                     color='polarity',
                     hover_name='Country',
                     custom_data=[avg_polarity_df['Country'], avg_polarity_df['polarity']],
                     projection='natural earth',
                     title='Geospatial Visualization of Average Polarity',
                     color_continuous_scale='RdBu_r',  # Adjust the color scale as needed
                     )

# Customize the layout
fig.update_geos(showcoastlines=True, coastlinecolor="Black", showland=True, landcolor="white")
fig.update_layout(coloraxis_colorbar=dict(title='Average Polarity'))

# Update hover data to display country name and polarity
fig.update_traces(hovertemplate='<b>Country:</b> %{customdata[0]}<br><b>Average Polarity:</b> %{customdata[1]:.2f}<extra></extra>')

fig.show()

### Task 2.4

###### Subjectivity Analysis

In [ ]:
# write a function to generate subjectivity

def getTextSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

In [ ]:
# applying above subjectivity function on text column

bit_samp['subjectivity'] = bit_samp['text'].apply(getTextSubjectivity)
bit_samp.head()

<ipython-input-25-69c6b34ae807>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,location,Lat,Lon,polarity,subjectivity
40306,LolitaNola,united states,stock trader-earth dweller-truth seeker- wears...,2011-01-15 18:07:29,1333.0,1191.0,210.0,True,2021-02-22 18:03:12,follow let hunt bitcoins together get exciting...,No hashtags,Twitter Web App,False,"(United States, (39.7837304, -100.445882))",39.783730,-100.445882,0.300,0.80
155482,Sir Chasington💎🤲,utah,"Software engineer, code. #Bitcoin. Golf, gym, ...",2011-12-28 04:58:43,637.0,274.0,2735.0,True,2021-06-23 01:42:21,bitcoin token matches btc price buy bitcoin,"['Bitcoin', 'Bitcoin']",Twitter for iPhone,False,"(Utah, United States, (39.4225192, -111.714358))",39.422519,-111.714358,0.000,0.00
147867,bit.trail,gothem,Follow the money 🚀. #bitcoin #btc #eth #crypto...,2011-04-27 06:25:55,2549.0,2563.0,735.0,True,2021-06-23 08:05:04,analyst bitcoin cryptos want get rich need alt...,"['Altcoins', 'Bitcoin', 'BTCTradingView', 'DeFi']",dlvr.it,False,"(Gothem, Gotlands kommun, Gotlands län, 624 30...",57.577952,18.730293,0.375,0.75
156952,Galen Briggs,montrose pa,∞/21M… Retire early with me. “100% balls deep ...,2017-02-19 21:59:27,30.0,68.0,665.0,True,2021-06-23 00:40:59,anyone ever calculated many bitcoin actually g...,['BITCOIN'],Twitter for iPhone,False,"(Montrose, Susquehanna County, Pennsylvania, U...",41.833965,-75.877139,0.250,0.30
104315,Crypto News Exchange | #CNE | #CryptoNews,moon,"John Klerm, #Crypto #Alchemist 🧙‍♂️\n\nOwner o...",2020-05-19 09:03:12,648.0,179.0,23361.0,False,2021-04-18 09:50:00,vertex market p p trading platform low fees do...,"['doge', 'dogecoin', 'bitcoin', 'crypto', 'btc...",Twitter Web App,False,"(Muhu, Saare maakond, Eesti, (58.5959044, 23.2...",58.595904,23.219646,0.200,0.55


In [ ]:
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
bit_samp_copy = bit_samp.copy()

# Group by geographical location and calculate the average polarity
avg_subjectivity_df = bit_samp_copy.groupby(['Lat', 'Lon'])['subjectivity'].mean().reset_index()

# Convert Lat and Lon to strings to create a hashable 'location' tuple
avg_subjectivity_df['location'] = avg_subjectivity_df[['Lat', 'Lon']].astype(str).apply(tuple, axis=1)

# Function to get country name from coordinates
def get_country(lat, lon):
    geolocator = Nominatim(user_agent="geo_analysis")
    location = None
    try:
        location = geolocator.reverse((lat, lon), language='en')
    except GeocoderTimedOut:
        pass

    return location.address.split(",")[-1].strip() if location else None

# Apply the function to create 'Country' column
avg_subjectivity_df['Country'] = avg_subjectivity_df.apply(lambda row: get_country(row['Lat'], row['Lon']), axis=1)


# Create a choropleth map for subjectivity
fig = px.scatter_geo(avg_subjectivity_df,
                     lat='Lat',
                     lon='Lon',
                     color='subjectivity',
                     hover_name='location',
                     custom_data=[avg_subjectivity_df['Country'],avg_subjectivity_df['subjectivity']],
                     projection='natural earth',
                     title='Geospatial Visualization of Average Subjectivity',
                     color_continuous_scale='RdBu_r',  # Adjust the color scale as needed
                     )

# Customize the layout
fig.update_geos(showcoastlines=True, coastlinecolor="Black", showland=True, landcolor="white")
fig.update_layout(coloraxis_colorbar=dict(title='Average Subjectivity'))

# Update hover data to display country name and polarity
fig.update_traces(hovertemplate='<b>Country:</b> %{customdata[0]}<br><b>Average Subjectivity:</b> %{customdata[1]:.2f}<extra></extra>')

fig.show()

###### Further analysis by identify influential users sentiments and highest and lowest polarity recorded countries

In [ ]:
# Explore user demographics
user_demographics = bit_samp[['user_name', 'user_location', 'user_description', 'user_created', 'user_followers', 'user_friends', 'user_verified']]

# Identify influential users based on the number of followers
influential_users = user_demographics[user_demographics['user_followers'] > user_demographics['user_followers'].quantile(0.95)]

# Analyze the sentiments expressed by influential users
influential_users_sentiments = bit_samp[bit_samp['user_name'].isin(influential_users['user_name'])]

In [ ]:
# Create a DataFrame with hover information
hover_info = influential_users_sentiments[['user_name', 'text', 'polarity', 'subjectivity']]

# Scatter plot
fig = px.scatter(
    hover_info,
    x='polarity',
    y='subjectivity',
    color='polarity',
    color_continuous_scale='RdBu',
    hover_data={'user_name': True, 'text': True, 'polarity': ':.2f', 'subjectivity': ':.2f'},
    title='Sentiments of Influential Users'
)

# Customize the layout
fig.update_layout(
    xaxis_title='Polarity',
    yaxis_title='Subjectivity',
    coloraxis_colorbar=dict(title='Polarity'),
    height=600,
    width=800
)

fig.show()

In [ ]:
# Calculate the percentage of neutral values in 'polarity'
neutral_polarity_percentage = (bit_samp['polarity'] == 0).mean() * 100

# Calculate the percentage of neutral values in 'subjectivity'
neutral_subjectivity_percentage = (bit_samp['subjectivity'] == 0).mean() * 100

# Print the percentages
print(f"Percentage of Neutral Polarity: {neutral_polarity_percentage:.2f}%")
print(f"Percentage of Neutral Subjectivity: {neutral_subjectivity_percentage:.2f}%")

Percentage of Neutral Polarity: 47.67%
Percentage of Neutral Subjectivity: 38.57%


### Task 2.5

###### Storify/Interpretation

As a policy advisor to the UK government,  geospatial data analysis reveals valuable insights into the public sentiment regarding cryptocurrency across the globe. The results are derived from some random tweets from bitcoins raw dataset.

The consistency in positive polarity across countries in America, East Europe, and Africa implies a global trend of favorable sentiments towards cryptocurrency.

High subjectivity in countries like the USA, Canada, Brazil, and Peru suggests that discussions around cryptocurrency are diverse and may involve varying perspectives, possibly influenced by factors such as economic considerations, technological awareness, or regulatory discussions.

The public opinion about cryptocurrency is nuanced and context dependent. As a policy advisor, it's crucial to recognize the diversity of opinions and tailor communication strategies accordingly.

Positive sentiments may warrant proactive support and collaboration, whereas regions with mixed or negative sentiments necessitate targeted efforts to address concerns and build trust.

Additionally, the moderate subjectivity levels indicate an opportunity for informed and balanced public discourse on cryptocurrency-related matters.

A substantial number of countries fall into the neutral category, indicating a mix of positive and negative sentiments. This suggests a global conversation marked by a variety of perspectives, debates, and uncertainties surrounding cryptocurrency.

Regional sentiment analysis unveils patterns that could be vital for understanding global sentiment clusters. Recognizing similarities in sentiment among neighbouring countries offers an opportunity to tailor region-specific policies that consider shared cultural, economic, and regulatory contexts.

The overall sentiment in the UK appears balanced, with a mix of neutral, positive, and a relatively small amount of negative sentiment.

The tweet with a polarity score of -0.40 stands out as more negative. Investigating the content of this tweet and understanding the context could provide insights into specific concerns or criticisms related to cryptocurrency.

Given the mix of sentiments, there may be opportunities to engage with the public, address concerns, and promote positive aspects of cryptocurrency.

Influential users, such as Brett Murphy, The Bitcoin News, and DYOR.net, play a significant role in shaping the narrative. Their followers, spanning from the United States to Nigeria, contribute to the amplification of sentiments, making it crucial to consider the impact of these influencers on public discourse.

The prevalence of neutral polarity (47.75%) suggests that a considerable portion of the discourse maintains a balanced stance or lacks strong emotional expression. This neutrality could stem from a variety of factors, including informational tweets, news sharing, or a cautious approach towards forming opinions.

The information retrieved from the sentiment analysis serves as a valuable tool for policymakers to gauge public perceptions, identify influential voices, and tailor region-specific approaches to cryptocurrency-related policies.

Continuous monitoring, engagement with influential users, and a nuanced understanding of sentiment dynamics will contribute to more informed decision-making in the evolving landscape of cryptocurrency regulation.
